In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
fml = pd.read_csv('drive/My Drive/twss/fml.txt',sep="\n",header=None)
fml.columns=['text']
fml.head()

,text
0,I downloaded an application for my phone that ...
1,"My girlfriend said ""It's funny how every time ..."
2,My girlfriend told me she was bored. During sex.
3,"I have a condition that, when I pull my foresk..."
4,My boyfriend decided to give me a hickey. Too ...


In [0]:
tfln = pd.read_csv('drive/My Drive/twss/tfln.onesent.txt',sep="\n",header=None,encoding='latin')
tfln.columns=['text']
tfln.head()

,text
0,she pulled the sheets over her head to blow me...
1,He just ordered a bottle of Beam at an Italian...
2,Well if my looks don't work with her I'll eat ...
3,"If you don't remember anything tomorrow, this ..."
4,We decided to cut you off after you insisted o...


In [0]:
usa = pd.read_csv('drive/My Drive/twss/usaquotes.txt',sep="\n",header=None,error_bad_lines=False)
usa.columns = ['text']
usa.head()

b'Skipping line 580: expected 1 fields, saw 2\nSkipping line 627: expected 1 fields, saw 2\nSkipping line 639: expected 1 fields, saw 2\nSkipping line 646: expected 1 fields, saw 2\nSkipping line 650: expected 1 fields, saw 2\nSkipping line 651: expected 1 fields, saw 3\nSkipping line 733: expected 1 fields, saw 6\nSkipping line 790: expected 1 fields, saw 2\nSkipping line 837: expected 1 fields, saw 4\nSkipping line 851: expected 1 fields, saw 2\nSkipping line 920: expected 1 fields, saw 3\nSkipping line 977: expected 1 fields, saw 2\nSkipping line 989: expected 1 fields, saw 2\nSkipping line 1245: expected 1 fields, saw 2\nSkipping line 1626: expected 1 fields, saw 3\nSkipping line 1796: expected 1 fields, saw 2\nSkipping line 1831: expected 1 fields, saw 3\nSkipping line 2021: expected 1 fields, saw 2\nSkipping line 2125: expected 1 fields, saw 2\nSkipping line 2206: expected 1 fields, saw 2\nSkipping line 2211: expected 1 fields, saw 2\nSkipping line 2624: expected 1 fields, saw 2\

,text
0,"We have too many high sounding words, and too ..."
1,The reins of government have been so long slac...
2,Shall we be despised by foreign powers for hes...
3,Deliver me from your cold phlegmatic preachers...
4,I regret the narrow contracted education of th...


In [0]:
twss = pd.read_csv('drive/My Drive/twss/twssstories.txt',sep="\n",header=None,encoding='latin')
twss.columns=['text']
twss.head()

,text
0,"Put it back in, it's still moist in the middle."
1,"Now if I can just get it in the hole, I'll sho..."
2,...a great penetration!
3,"That just kind of squirted out there, didn't it!"
4,"Oh it feels so good, you just insert here and ..."


In [0]:
print("fml : ",len(fml))
print("tfln : ",len(tfln))
print("usa : ",len(usa))
print("twss : ",len(twss))

fml :  1767
tfln :  11869
usa :  4066
twss :  2027


In [0]:
twss['target'] = [1]*2027
twss.head()

,text,target
0,"Put it back in, it's still moist in the middle.",1
1,"Now if I can just get it in the hole, I'll sho...",1
2,...a great penetration!,1
3,"That just kind of squirted out there, didn't it!",1
4,"Oh it feels so good, you just insert here and ...",1


In [0]:
usa = usa.loc[:666,:]
tfln = tfln.loc[:666,:]
fml = fml.loc[:666,:]

In [0]:
usa['target'] = [0]*667

In [0]:
tfln['target'] = [0]*667
fml['target'] = [0]*667

In [0]:
df = pd.concat([twss,usa,tfln,fml],ignore_index=True)

In [0]:
df

,text,target
0,"Put it back in, it's still moist in the middle.",1
1,"Now if I can just get it in the hole, I'll sho...",1
2,...a great penetration!,1
3,"That just kind of squirted out there, didn't it!",1
4,"Oh it feels so good, you just insert here and ...",1
...,...,...
4023,While my wife was watching me get undressed sh...,0
4024,My boyfriend came to my workplace and presente...,0
4025,"I was laying on the bed, naked, waiting for my...",0
4026,I hung out with the guy I've liked for the fir...,0


In [0]:
df = df.sample(frac=1).reset_index(drop=True)

In [0]:
df

,text,target
0,It's sooo cute when you nibble it softly like ...,1
1,I finally felt ready to take my bra off during...,0
2,KEVIN! Stop slapping it like that! Just smooth...,1
3,I cant yet im literally covered in lube but I ...,0
4,"My ass is really sore, but I like it a lot.",1
...,...,...
4023,I know I'm her Sunday school teacher. I just f...,0
4024,I'm now at that point where it just feels natu...,0
4025,You just begged me to mute the porn and watch ...,0
4026,There are two thoughts that will ensure succes...,0


In [0]:
y = df['target']
df.drop(['target'],inplace=True,axis=1)

In [0]:

xtrain,xvalid,ytrain,yvalid= train_test_split(df.text.values,y,stratify=y,random_state=42,test_size=0.1,shuffle=True)

In [0]:
print(xtrain.shape,xvalid.shape)

(3625,) (403,)


In [0]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)


In [0]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict(xvalid_tfv)

print(metrics.accuracy_score(predictions,yvalid))

0.9181141439205955


In [0]:
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict(xvalid_tfv)

print(metrics.accuracy_score(predictions,yvalid))

0.9330024813895782


In [0]:
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict(xvalid_svd_scl)

print(metrics.accuracy_score(predictions,yvalid))

0.913151364764268


In [0]:
import xgboost

xgb = xgboost.XGBClassifier(n_thread=10)
xgb.fit(xtrain_tfv,ytrain)

predictions = xgb.predict(xvalid_tfv)

print(metrics.accuracy_score(predictions,yvalid))

0.8287841191066998


In [0]:
xgb.fit(xtrain_svd,ytrain)

predictions = xgb.predict(xvalid_svd)

print(metrics.accuracy_score(predictions,yvalid))

0.8709677419354839


In [0]:


mll_scorer = metrics.make_scorer(metrics.accuracy_score, greater_is_better=True, needs_proba=False)



nb_model = MultinomialNB()

# Create the pipeline 
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
Best score: 0.898
Best parameters set:
	nb__alpha: 0.1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0163s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0302s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.1s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


In [0]:
clf = MultinomialNB(alpha=0.1)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict(xvalid_tfv)

print(metrics.accuracy_score(predictions,yvalid))

0.9404466501240695


In [0]:
def tfconverter(x):
  tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')
  tfv.fit(list(xtrain) + list(xvalid))

  return tfv.transform(x)
  


In [0]:
x = tfconverter([input()])
clf = MultinomialNB(alpha=0.1)
clf.fit(xtrain_tfv, ytrain)
res = clf.predict(x)

if res == np.array([0]):
  print("Nope")
else:
  print("That's what she said")



i cant take it anymore
That's what she said


In [51]:
clf

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [0]:
import pickle
pickle.dump(clf,open('model.pkl','wb'))


In [0]:
model = pickle.load(open('model.pkl','rb'))

In [56]:
x = tfconverter([input()])
model.predict(x)

that's the wrong hole


array([1])